In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import preprocess_temp as P
import Model as M

### Load Data

In [3]:
directory = './conala-corpus/'
train_file = directory + 'train.json'
test_file = directory + 'test.json'

with open(train_file) as f:
    train_data = json.load(f)
    
with open(test_file) as f:
    test_data = json.load(f)

### Let's preprocess the data. Everything is in Preprocess.py

In [4]:
# intent processing includes lowercase, remove punctuation'?'
train_intent, train_codes = P.process_data(train_data)
test_intent, test_codes = P.process_data(test_data)

In [6]:
# this class is used for code2actions and actions2code
ast_action = P.Ast_Action()

In [7]:
train_actions = []

for code in train_codes:
    train_actions.append(ast_action.code2actions(code))

In [8]:
word_lst = P.vocab_list(train_intent, cut_freq=5)
act_lst, token_lst = P.action_list(train_actions, cut_freq=5)

In [9]:
word2num = dict(zip(word_lst, range(0,len(word_lst))))
act2num = dict(zip(act_lst, range(0,len(act_lst))))
token2num = dict(zip(token_lst, range(0,len(token_lst))))

In [23]:
train_loader = P.get_train_loader(train_intent, train_actions, word2num, act2num, token2num)

In [26]:
test_loader = P.get_test_loader(test_intent, word2num)

### A demonstration of Actions to code

In [11]:
code = '''print('asdf;lkhdfg')'''

In [13]:
code_actions = ast_action.code2actions(code)

In [26]:
nums = []
for action in code_actions:
    if action in action2num:
        nums.append(action2num[action])
    else:
        nums.append(action2num[P.GenTokenAction('</UNK>')])

In [27]:
print(nums)

[0, 1, 2, 64, 26, 385, 28, 14, 14]


In [28]:
actions = []
for num in nums:
    actions.append(num2action[num])

In [29]:
ast_action.actions2code(actions)

"print('</UNK>')"

### Model